In [1]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version:{}".format(sys.version))
print("Tensorflow version:{}".format(tf.__version__))

System version:3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]
Tensorflow version:2.6.1


In [ ]:
import numpy as np

In [ ]:
!pip install keras==2.6

In [ ]:
!pip install protobuf==3.20.* --user

In [ ]:
!pip install scrapbook

In [ ]:
!pip install tensorflow==2.6.1

In [ ]:
!pip install numpy==1.20.3 --user

In [ ]:
conda uninstall numpy

In [ ]:
!pip show numpy

In [15]:
import pandas as pd
import random
import numpy as np

In [16]:
df1=pd.read_table('traces.tsv')
df1

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
0,222387,2009-10-04 12:33:26,19913,1,Becker1990,0,12,0,0,0,0,0
1,222387,2009-10-04 14:28:14,13029,2,Gaynor66,0,24,4,0,0,1,0
2,222387,2009-10-04 14:44:08,12079,3,Jostrem16,0,36,4,0,0,2,0
3,222387,2009-10-04 17:14:26,3071,4,Clubfan78,0,48,10,0,0,3,0
4,222387,2009-10-04 17:36:06,1773,5,Momstired2,0,60,2,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2049967,259070,2009-12-12 21:29:30,15,517,Blaufish83,0,3102,14,0,1,374,0
2049968,259070,2009-12-12 21:29:30,15,518,Solwicca,0,3108,14,0,1,374,0
2049969,259070,2009-12-12 21:29:30,15,519,Jaden248,0,3114,14,0,1,374,0
2049970,259070,2009-12-12 21:29:30,15,520,Orion29,0,3120,14,0,0,374,0


In [17]:
df2=df1.drop_duplicates(subset=['auction_id','bid_user']).groupby('bid_user').size().sort_values(ascending=False).reset_index()
df2.columns =['bid_user', 'counts']
df2

,bid_user,counts
0,Lillyblitz,258
1,Npr1967,225
2,Drodenbeck,214
3,Zwerg1953,191
4,Varis,191
...,...,...
60532,Keebler87,1
60533,Kedii,1
60534,Kebo178,1
60535,Keber314,1


In [18]:
df2=df2[df2['counts']>=2]
df2

,bid_user,counts
0,Lillyblitz,258
1,Npr1967,225
2,Drodenbeck,214
3,Zwerg1953,191
4,Varis,191
...,...,...
37244,Dansgr,2
37245,Julsche,2
37246,Topdolly75,2
37247,Topazlady,2


In [19]:
df3=df1[df1['bid_user'].isin(df2['bid_user'].tolist())]
df3

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
0,222387,2009-10-04 12:33:26,19913,1,Becker1990,0,12,0,0,0,0,0
1,222387,2009-10-04 14:28:14,13029,2,Gaynor66,0,24,4,0,0,1,0
2,222387,2009-10-04 14:44:08,12079,3,Jostrem16,0,36,4,0,0,2,0
3,222387,2009-10-04 17:14:26,3071,4,Clubfan78,0,48,10,0,0,3,0
4,222387,2009-10-04 17:36:06,1773,5,Momstired2,0,60,2,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2049967,259070,2009-12-12 21:29:30,15,517,Blaufish83,0,3102,14,0,1,374,0
2049968,259070,2009-12-12 21:29:30,15,518,Solwicca,0,3108,14,0,1,374,0
2049969,259070,2009-12-12 21:29:30,15,519,Jaden248,0,3114,14,0,1,374,0
2049970,259070,2009-12-12 21:29:30,15,520,Orion29,0,3120,14,0,0,374,0


In [20]:
df4=df3.drop_duplicates(subset=['auction_id','bid_user']).groupby('bid_user').size().sort_values(ascending=False)#reset_index()
df4

bid_user
Lillyblitz     258
Npr1967        225
Drodenbeck     214
Alad99         191
Varis          191
              ... 
Paroca           2
Dpauken73        2
Parotgal         2
Partial          2
Queenb4u2nv      2
Length: 37249, dtype: int64

In [21]:
df5=df3.sort_values('bid_time')
df5=df5.drop_duplicates(subset=['auction_id','bid_user'])
df5

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
19003,223829,2009-10-01 22:07:40,2854,1,Sash6,0,12,0,0,0,0,0
19006,223829,2009-10-01 22:51:21,248,4,Rfire,0,48,2,0,0,3,0
19007,223829,2009-10-01 22:52:14,197,5,dome60,0,60,2,0,0,4,0
19014,223829,2009-10-01 22:55:53,14,12,Yllekbryan,0,144,0,0,1,9,0
3924,222653,2009-10-02 03:17:59,36737,1,Rvs66,0,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2043476,259045,2009-12-12 22:30:36,15,10,Tomr12766,0,60,0,0,1,6,0
2043477,259045,2009-12-12 22:30:36,15,11,Gapkiddo,0,66,0,0,1,6,0
2043478,259045,2009-12-12 22:30:36,15,12,1940neo,0,72,0,0,1,6,0
2043480,259045,2009-12-12 22:30:36,15,14,Dram18,0,84,0,0,0,6,0


In [23]:
df5['auction_id']=df5['auction_id'].astype(str)

In [24]:
df5

,auction_id,bid_time,bid_ct,bid_number,bid_user,bid_butler,bid_cp,bid_user_secs_added,bid_butler_secs_added,bid_infered,bid_group,bid_final
19003,223829,2009-10-01 22:07:40,2854,1,Sash6,0,12,0,0,0,0,0
19006,223829,2009-10-01 22:51:21,248,4,Rfire,0,48,2,0,0,3,0
19007,223829,2009-10-01 22:52:14,197,5,dome60,0,60,2,0,0,4,0
19014,223829,2009-10-01 22:55:53,14,12,Yllekbryan,0,144,0,0,1,9,0
3924,222653,2009-10-02 03:17:59,36737,1,Rvs66,0,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2043476,259045,2009-12-12 22:30:36,15,10,Tomr12766,0,60,0,0,1,6,0
2043477,259045,2009-12-12 22:30:36,15,11,Gapkiddo,0,66,0,0,1,6,0
2043478,259045,2009-12-12 22:30:36,15,12,1940neo,0,72,0,0,1,6,0
2043480,259045,2009-12-12 22:30:36,15,14,Dram18,0,84,0,0,0,6,0


In [25]:
impressionData=df5.groupby('bid_user')['auction_id'].apply(" ".join).reset_index()
print(impressionData)

              bid_user                                         auction_id
0         +34661281XXX                        224875 224929 223626 225739
1         +34938723XXX                                      247448 247083
2         +34976394XXX                                      239006 240958
3        +436604727XXX                                      231525 243429
4        +436606822XXX                        229731 248057 249875 250942
...                ...                                                ...
37244          zarte28  224026 225028 227526 225181 225553 225885 2251...
37245           zate28                        250507 250042 248768 248639
37246         ziomekpl                                      247712 247018
37247  zitronenfalter7                                      252311 255969
37248          zorro54                               239349 239339 259044

[37249 rows x 2 columns]


In [26]:
impressionData['auction_id'].dtype

dtype('O')

In [27]:
def noLastOne(text):
  words=text.split()
  if len(words)<=1:
    return ''
  else:
    return ' '.join(words[-51:-1])

def keepLastOne(text):
  newWords=text.split()
  if len(newWords)<1:
    return ''
  else:
     impressionWords=[newWords[-1]]+[str(random.randint(100000, 999999))for _ in range(20)]
     random_index = random.randint(0,len(impressionWords)-1)
     impressionWords[random_index]+='-1'
     return ' '.join(impressionWords)

impressionData['history']=impressionData['auction_id'].apply(noLastOne)
impressionData['impression']=impressionData['auction_id'].apply(keepLastOne)

print(impressionData)

              bid_user                                         auction_id  \
0         +34661281XXX                        224875 224929 223626 225739   
1         +34938723XXX                                      247448 247083   
2         +34976394XXX                                      239006 240958   
3        +436604727XXX                                      231525 243429   
4        +436606822XXX                        229731 248057 249875 250942   
...                ...                                                ...   
37244          zarte28  224026 225028 227526 225181 225553 225885 2251...   
37245           zate28                        250507 250042 248768 248639   
37246         ziomekpl                                      247712 247018   
37247  zitronenfalter7                                      252311 255969   
37248          zorro54                               239349 239339 259044   

                                                history  \
0               

In [29]:
impression2=impressionData
impression2

,bid_user,auction_id,history,impression
0,+34661281XXX,224875 224929 223626 225739,224875 224929 223626,225739 706101 385336 552169 321841 895131 2552...
1,+34938723XXX,247448 247083,247448,247083 458088 730178 330421 951720 179023 4031...
2,+34976394XXX,239006 240958,239006,240958 274469 208818 508829 822611 836230 1189...
3,+436604727XXX,231525 243429,231525,243429 171838 781587 517574 674938-1 416531 29...
4,+436606822XXX,229731 248057 249875 250942,229731 248057 249875,250942-1 668384 619031 865078 741383 384204 26...
...,...,...,...,...
37244,zarte28,224026 225028 227526 225181 225553 225885 2251...,224026 225028 227526 225181 225553 225885 225115,226731 321941 400101 400664 672267-1 924717 86...
37245,zate28,250507 250042 248768 248639,250507 250042 248768,248639 873115 961549 837868 784114-1 814120 23...
37246,ziomekpl,247712 247018,247712,247018 413090 921386 706598 485219 588222 7543...
37247,zitronenfalter7,252311 255969,252311,255969-1 536684 693536 718527 297036 261746 92...


In [30]:
def addZero(text):
    words=text.split()
    newWords=[]
    for word in words:
        if '-1' not in word:
            word+='-0'
        newWords.append(word)

    return ' '.join(newWords)

impression2['impression']=impression2['impression'].apply(addZero)

print(impression2)

              bid_user                                         auction_id  \
0         +34661281XXX                        224875 224929 223626 225739   
1         +34938723XXX                                      247448 247083   
2         +34976394XXX                                      239006 240958   
3        +436604727XXX                                      231525 243429   
4        +436606822XXX                        229731 248057 249875 250942   
...                ...                                                ...   
37244          zarte28  224026 225028 227526 225181 225553 225885 2251...   
37245           zate28                        250507 250042 248768 248639   
37246         ziomekpl                                      247712 247018   
37247  zitronenfalter7                                      252311 255969   
37248          zorro54                               239349 239339 259044   

                                                history  \
0               

In [31]:
impression3=impression2.drop('auction_id',axis='columns')
impression3

,bid_user,history,impression
0,+34661281XXX,224875 224929 223626,225739-0 706101-0 385336-0 552169-0 321841-0 8...
1,+34938723XXX,247448,247083-0 458088-0 730178-0 330421-0 951720-0 1...
2,+34976394XXX,239006,240958-0 274469-0 208818-0 508829-0 822611-0 8...
3,+436604727XXX,231525,243429-0 171838-0 781587-0 517574-0 674938-1 4...
4,+436606822XXX,229731 248057 249875,250942-1 668384-0 619031-0 865078-0 741383-0 3...
...,...,...,...
37244,zarte28,224026 225028 227526 225181 225553 225885 225115,226731-0 321941-0 400101-0 400664-0 672267-1 9...
37245,zate28,250507 250042 248768,248639-0 873115-0 961549-0 837868-0 784114-1 8...
37246,ziomekpl,247712,247018-0 413090-0 921386-0 706598-0 485219-0 5...
37247,zitronenfalter7,252311,255969-1 536684-0 693536-0 718527-0 297036-0 2...


In [35]:
impression4=impression3.head(100)
impression4

,bid_user,history,impression
0,+34661281XXX,224875 224929 223626,225739-0 706101-0 385336-0 552169-0 321841-0 8...
1,+34938723XXX,247448,247083-0 458088-0 730178-0 330421-0 951720-0 1...
2,+34976394XXX,239006,240958-0 274469-0 208818-0 508829-0 822611-0 8...
3,+436604727XXX,231525,243429-0 171838-0 781587-0 517574-0 674938-1 4...
4,+436606822XXX,229731 248057 249875,250942-1 668384-0 619031-0 865078-0 741383-0 3...
...,...,...,...
95,+492065907XXX,226084,226094-0 925116-0 677688-0 771466-0 669259-0 8...
96,+4921117806XXX,230867,237520-0 521470-0 734870-0 957762-0 495261-0 5...
97,+492119008XXX,249812,250015-1 596674-0 338899-0 780485-0 206407-0 6...
98,+4921374XXX,229769,227701-0 967117-0 479579-0 622278-0 974068-0 7...


In [32]:
df=pd.read_table('outcomes.tsv')
df.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,placedbids,freebids,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,15,75,Racer11,26,0,19:52 PDT 09-16-2008,0,0,0,0
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,15,75,Cemo23,65,0,11:17 PDT 08-28-2008,0,0,0,0
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,15,75,Jacobsonnich,94,0,22:52 PDT 11-07-2008,0,1,0,0
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,15,75,Mokkis,10,0,22:02 PDT 08-23-2008,0,0,0,0
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,15,75,Superloeffel,80,0,14:23 PDT 08-24-2008,0,0,0,0


In [34]:
df10=df.drop(['product_id','item','retail','price','finalprice','bidincrement','bidfee','winner','placedbids','freebids','endtime_str','flg_click_only','flg_beginnerauction','flg_fixedprice','flg_endprice'],axis='columns')
df10

,auction_id,desc
0,86827,Sony Ericsson S500i Unlocked Mysterious Green
1,87964,PSP Slim & Lite Sony Piano Black
2,87965,PSP Slim & Lite Sony Piano Black
3,88638,Sony Ericsson S500i Unlocked Mysterious Green
4,88639,Sony Ericsson S500i Unlocked Mysterious Green
...,...,...
121414,259065,300 Bids Voucher
121415,259066,300 Bids Voucher
121416,259068,300 Bids Voucher
121417,259069,300 Bids Voucher


In [36]:
df11=df10.head(100)
df11

,auction_id,desc
0,86827,Sony Ericsson S500i Unlocked Mysterious Green
1,87964,PSP Slim & Lite Sony Piano Black
2,87965,PSP Slim & Lite Sony Piano Black
3,88638,Sony Ericsson S500i Unlocked Mysterious Green
4,88639,Sony Ericsson S500i Unlocked Mysterious Green
...,...,...
95,93281,SanDisk Cruzer Contour 4GB
96,93282,SanDisk Cruzer Contour 4GB
97,93283,SanDisk Cruzer Contour 4GB
98,93284,SanDisk Cruzer Contour 4GB


In [11]:
epochs=1
seed=40
batch_size=64

In [6]:
MIND_type='demo'

In [40]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', df11)
train_behaviors_file = os.path.join(data_path, 'train', impression4)
valid_news_file = os.path.join(data_path, 'valid', df11)
valid_behaviors_file = os.path.join(data_path, 'valid', impression4)
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'DataFrame'

In [12]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 1, 'batch_size': 64, 'show_step': 100000, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'lstur', 'loss': 'cross_entropy_loss', 'wordEmb_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmp1xlt7xl2\\utils\\embedding.npy', 'wordDict_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmp1xlt7xl2\\utils\\word_dict.pkl', 'userDict_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmp1xlt7xl2\\utils\\uid2index.pkl'}


In [14]:
iterator = MINDIterator

In [38]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d/Relu:0", shape=(None, 30, 400), dtype=float32)
Tensor("att_layer2/Sum_1:0", shape=(None, 400), dtype=float32)


C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [39]:
print(model.run_eval(df11, impression4))

0it [00:00, ?it/s]


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. tensorflow.python.lib.io._pywrap_file_io.BufferedInputStream(filename: str, buffer_size: int, token: tensorflow.python.lib.io._pywrap_file_io.TransactionToken = None)

Invoked with:     auction_id                                           desc
0        86827  Sony Ericsson S500i Unlocked Mysterious Green
1        87964               PSP Slim & Lite Sony Piano Black
2        87965               PSP Slim & Lite Sony Piano Black
3        88638  Sony Ericsson S500i Unlocked Mysterious Green
4        88639  Sony Ericsson S500i Unlocked Mysterious Green
..         ...                                            ...
95       93281                     SanDisk Cruzer Contour 4GB
96       93282                     SanDisk Cruzer Contour 4GB
97       93283                     SanDisk Cruzer Contour 4GB
98       93284                     SanDisk Cruzer Contour 4GB
99       93285            